In [3]:
pip install pyathena

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.0 MB/s eta 0:00:00ta 0:00:01
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 778.5 kB/s eta 0:00:00a 0:00:01
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.0.7 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyathena import connect
import pandas as pd

In [6]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [7]:
#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')

df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_352/3368215165.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [8]:
df = df[df['amount'] != 0]

In [9]:
df.describe()

,tx,amount,coupon_count
count,93018.000000,9.301800e+04,93018.000000
mean,588.385409,2.699545e+05,72.697682
std,1867.914359,8.941920e+05,378.282519
min,1.000000,1.000000e+00,0.000000
25%,13.000000,5.394250e+03,0.000000
50%,91.000000,3.552221e+04,2.000000
75%,442.000000,1.742184e+05,17.000000
max,29203.000000,1.391208e+07,11564.000000


In [13]:
df['date'] = pd.to_datetime(df['date'])

In [14]:
# Adding 'week' column >  week number
df['week'] = df['date'].dt.strftime('%Y-%U')

In [15]:
#Sorting dates
df = df.sort_values(by='date')

In [16]:
# Adding  'day_of_week' column
df['day_of_week'] = df['date'].dt.day_name()

In [17]:
# We are defining a dynamic timeframe (7 days, NOT a week)
# Calculating statistics for the rolling time window
rolling_stats = df.groupby(['country', 'payer']).rolling(window=7, on='date')['amount'].agg(['mean', 'min', 'max', 'var']).reset_index()


In [18]:
# Merge the original DataFrame with the rolling window statistics
df_amount = pd.merge(df, rolling_stats, on=['date','country', 'payer' ])

In [19]:
df_amount.columns

Index(['payer', 'country', 'date', 'tx', 'amount', 'coupon_count', 'week',
       'day_of_week', 'mean', 'min', 'max', 'var'],
      dtype='object')

In [58]:
#Warning! Dropping null dates
df_amount = df_amount[['date', 'country','payer', 'amount', 'day_of_week', 'mean', 'min', 'max', 'var']].dropna()

In [20]:
df_amount.sort_values(['date','country','payer'], ascending=False)

,payer,country,date,tx,amount,coupon_count,week,day_of_week,mean,min,max,var
92916,VIAMERICAS USA DEPOSITS,UNITED STATES,2023-10-21,197,118954.15,0,2023-42,Saturday,120236.097143,80851.87,151268.00,5.402184e+08
93005,MASTERCARD ATM NETWORK,UNITED STATES,2023-10-21,11,2340.00,0,2023-42,Saturday,2934.285714,1400.00,4620.00,1.118629e+06
92915,NAFA,TOGO,2023-10-21,4,1932.00,0,2023-42,Saturday,3668.474286,170.00,11048.32,1.367871e+07
93016,TRANSPAY,THAILAND,2023-10-21,1,500.00,0,2023-42,Saturday,NaN,NaN,NaN,NaN
92966,TRANSFERTO - THUNES,SPAIN,2023-10-21,22,8896.75,0,2023-42,Saturday,5375.652857,3449.99,8896.75,3.315774e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
30,BANCO RENDIMENTO,BRAZIL,2020-12-31,372,225378.87,375,2020-52,Thursday,NaN,NaN,NaN,NaN
14,BANCO DAYCOVAL,BRAZIL,2020-12-31,85,36333.46,86,2020-52,Thursday,NaN,NaN,NaN,NaN
17,BANCO GANADERO (BOLIVIA),BOLIVIA,2020-12-31,10,1582.00,9,2020-52,Thursday,NaN,NaN,NaN,NaN
22,BANCO BISA,BOLIVIA,2020-12-31,6,1360.00,6,2020-52,Thursday,NaN,NaN,NaN,NaN


In [21]:
df_amount[df_amount['country'] == 'MEXICO'].sort_values(['date','payer'], ascending=False)

,payer,country,date,tx,amount,coupon_count,week,day_of_week,mean,min,max,var
92982,WALMART (UT),MEXICO,2023-10-21,1617,625157.45,29,2023-42,Saturday,429818.541429,282912.89,625157.45,1.782442e+10
92987,WALDOS (APPRIZA),MEXICO,2023-10-21,76,22712.00,2,2023-42,Saturday,10152.777143,5028.00,22712.00,3.978767e+07
92912,TRANSFER DIRECTO,MEXICO,2023-10-21,187,121276.76,2,2023-42,Saturday,114752.860000,102802.44,139234.08,1.662581e+08
92932,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,2023-10-21,992,591126.82,22,2023-42,Saturday,578143.514286,462226.37,734867.53,1.035160e+10
93002,SORIANA,MEXICO,2023-10-21,819,267729.48,2,2023-42,Saturday,184892.552857,128042.68,267729.48,3.569255e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
37,BANSEFI - BANCO DEL BIENESTAR,MEXICO,2020-12-31,220,106964.84,177,2020-52,Thursday,NaN,NaN,NaN,NaN
58,BANORTE (UT),MEXICO,2020-12-31,313,172432.93,242,2020-52,Thursday,NaN,NaN,NaN,NaN
71,BANCOPPEL (BTS),MEXICO,2020-12-31,4076,1309747.34,3219,2020-52,Thursday,NaN,NaN,NaN,NaN
77,AIRPAK,MEXICO,2020-12-31,63,20459.00,51,2020-52,Thursday,NaN,NaN,NaN,NaN


In [22]:
df_mex = df_amount[df_amount['country'] == 'MEXICO'].groupby

In [23]:
df_amount[['date', 'country','payer', 'amount', 'day_of_week', 'mean', 'min', 'max', 'var']].dropna()

,date,country,payer,amount,day_of_week,mean,min,max,var
472,2021-01-06,MEXICO,CAJA POPULAR MEXICANA (UT),28313.25,Wednesday,34237.721429,9130.00,56292.50,2.177701e+08
474,2021-01-06,HONDURAS,BANHCAFE,4242.00,Wednesday,2770.285714,850.00,5825.00,3.429796e+06
475,2021-01-06,HONDURAS,COOPERATIVA FRONTERIZA (UT),5809.00,Wednesday,5597.815714,600.00,9188.71,8.013406e+06
476,2021-01-06,ECUADOR,BANCO DE GUAYAQUIL,7770.00,Wednesday,12567.285714,250.00,20967.00,5.641717e+07
477,2021-01-06,ECUADOR,GLOBAL ENVIOS,20558.00,Wednesday,18266.214286,120.00,28914.00,9.672791e+07
...,...,...,...,...,...,...,...,...,...
93012,2023-10-21,CAMEROON,ZEEPAY,772.78,Saturday,1038.855714,437.00,2409.00,4.657211e+05
93013,2023-10-21,NEPAL,GME,975.00,Saturday,6409.652857,975.00,10981.45,1.480035e+07
93014,2023-10-21,BANGLADESH,MERCHANTRADE,1608.00,Saturday,928.000000,5.00,2226.00,8.144960e+05
93015,2023-10-21,NIGERIA,NAIRAGRAM,82268.62,Saturday,83164.785714,8536.17,122077.44,1.329003e+09
